In [194]:
import pandas as pd

In [5]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

### Чтение таблиц с диска

In [196]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of spreadsheets
SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'
CLIENTS_RANGE = 'clients!A1:C74766'
LEADS_RANGE = 'leads!A1:F3338'
MANAGERS_RANGE = 'managers!A1:C14'

def read_table(table_range):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Читаем таблицу
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=table_range).execute()
    table = result.get('values', [])
    
    # Преобразуем таблицу в датафрейм
    table_df = pd.DataFrame(table[1:], columns=table[0])
    return table_df

### Запись таблиц на диск

In [186]:
# The ID and range of a sample spreadsheet.
# "https://docs.google.com/spreadsheets/d/17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA/edit?usp=sharing"

WRITE_SPREADSHEET_ID = '17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA'

def write_table(table, rng):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Пишем таблицу
    sheet = service.spreadsheets()
    request = sheet.values().batchUpdate(spreadsheetId=WRITE_SPREADSHEET_ID, 
                                    body = {
                                        "valueInputOption": "RAW", 
                                        "data": [
                                            {"range": rng,
                                             "majorDimension": "ROWS",     
                                             "values": [table.columns.values.tolist()] + table.values.tolist()
                                            }
                                        ]
                                    })
    
    response = request.execute()
    print(response)

In [8]:
leads_df = read_table(LEADS_RANGE) # читаем таблицу с заявками
managers_df = read_table(MANAGERS_RANGE) # читаем таблицу с менеджерами

In [18]:
leads_df.head()

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1


### Создание Dimensions

In [197]:
# Получить d_manager по id менеджера
def get_d_manager_by_id(manager_id):
    not_manager_id = "00000000-0000-0000-0000-000000000000"
    if manager_id == not_manager_id:
        return "-"
    manager = managers_df[managers_df['manager_id'] == manager_id]['d_manager']
    if manager.empty:
        return "invalid id"
    else:
        return manager.iloc[0]

In [198]:
# Получить d_club по id менеджера
def get_d_club_by_id(manager_id):
    not_manager_id = "00000000-0000-0000-0000-000000000000"
    if manager_id == not_manager_id:
        return "-"
    club = managers_df[managers_df['manager_id'] == manager_id]['d_club']
    if club.empty:
        return "invalid id"
    else:
        return club.iloc[0]

In [213]:
dimensions = leads_df[["lead_id", "d_utm_source", "l_manager_id"]].copy() # копируем из leads только полезные столбцы

In [214]:
dimensions['d_manager'] = dimensions['l_manager_id'].apply(
                            lambda x: get_d_manager_by_id(x)) # подставить d_manager

In [215]:
dimensions['d_club'] = dimensions['l_manager_id'].apply(
                            lambda x: get_d_club_by_id(x)) # подставить d_club

In [216]:
dimensions = dimensions.drop(['l_manager_id'], axis=1) # удалить лишний столбец с индексами менеджеров

In [217]:
dimensions.head()

,lead_id,d_utm_source,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,viber,manager #8,club #2
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,instagram,manager #1,club #1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,instagram,manager #11,club #4
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,,-,-
4,009b1616-145d-11eb-ac1d-c412f533dba1,vk,manager #10,club #2


#### Запись таблицы Dimensions на диск

In [211]:
DIMENSIONS_RANGE = 'Dimensions!A1:D' + str(len(dimensions) + 1)
write_table(dimensions, DIMENSIONS_RANGE)

{'spreadsheetId': '17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA', 'totalUpdatedRows': 3338, 'totalUpdatedColumns': 4, 'totalUpdatedCells': 13352, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA', 'updatedRange': 'Dimensions!A1:D3338', 'updatedRows': 3338, 'updatedColumns': 4, 'updatedCells': 13352}]}


### Создание таблицы для вычисления Measures

#### 1. количество заявок

In [115]:
len(dimensions) # просто для проверки

3337

#### 2. количество мусорных заявок (на основании заявки не создан клиент)

In [219]:
CLIENTS_RANGE = "clients!A1:C75767"
clients_df = read_table(CLIENTS_RANGE) # мы еще не считали таблицу с клиентами - читаем

In [222]:
clients_df.head()

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1


Если сделаем подсчет значений, то обнаружим, что на каждого клиента может приходиться больше одной заявки. Тогда будем считать, что первая по времени заявка привела к созданию клиента, а остальные оказались "мусорными"

In [226]:
leads_df["l_client_id"].value_counts() 

00000000-0000-0000-0000-000000000000    824
c4447d16-29bf-11eb-ac1f-c412f533dba1      9
ea4a2fd6-3de4-11ea-abeb-c412f533dba1      6
f13f693a-1a02-11eb-ac1d-c412f533dba1      4
cdf6d7bb-e599-11e8-ab7d-c412f533dba1      4
                                       ... 
b2aac912-d258-11ea-ac0f-c412f533dba1      1
399e23e9-1c10-11eb-ac1d-c412f533dba1      1
c31e2aba-12cf-11eb-ac1c-c412f533dba1      1
237042b4-1541-11eb-ac1d-c412f533dba1      1
2aded335-1c41-11eb-ac1d-c412f533dba1      1
Name: l_client_id, Length: 2183, dtype: int64

А все ли клиенты из таблицы заявок попали в таблицу клиентов? Да

In [223]:
leads_df_client = set(leads_df["l_client_id"].unique())
clients = set(clients_df["client_id"].unique())
leads_df_client - clients

{'00000000-0000-0000-0000-000000000000'}

In [227]:
# функция, которая принимает решение, мусорная заявка или нет
def set_if_trash(row):
    counts = leads_df["l_client_id"].value_counts() 
    if row["l_client_id"] == "00000000-0000-0000-0000-000000000000": # если нулевые id, то одназначно мусорная
        return "trash"
    if counts[row["l_client_id"]] == 1: # если заявка встретилась один раз, тогда она точно привела к созданию клиента
        return "not trash"
    # иначе проверим, является ли эта заявка самой ранней
    # выберем из таблицы заявок все заявки этого клиента
    # отсортируем их по дате
    # обновим индексы в этом мини-датафрейме
    leads = leads_df[leads_df["l_client_id"] == row["l_client_id"]].sort_values(by="created_at").reset_index()
    # если наша заявка (проверим по id заявки) окажется первой в мини-датафреме, то она привела к созданию клиента
    if leads[leads["lead_id"] == row["lead_id"]].index[0] == 0:
        return "not trash"
    else:
        return "trash"

In [228]:
leads_df["is_trash"] = leads_df.apply(lambda row: set_if_trash(row), axis=1) # создадим столбец с пометкой о заявке

#### 3. количество новых заявок (не было заявок и покупок от этого клиента раньше)

In [103]:
TRANS_ID = "transactions!A1:D29001"
trans_df = read_table(TRANS_ID) # читаем таблицу танзакций

In [96]:
trans_df.head()

,transaction_id,created_at,m_real_amount,l_client_id
0,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,31,8a805e60-6fd6-11e7-80fc-c412f533dba1
1,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,87,8a805e60-6fd6-11e7-80fc-c412f533dba1
2,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,49231,dc0d0e52-629a-11ea-abf3-c412f533dba1
3,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,17305,dc0d0e52-629a-11ea-abf3-c412f533dba1
4,52f8ebcc-d82d-4be1-8004-72cfbe0dff24,2020-09-25 06:00:07,9022,e663b6a0-4a1a-11e8-a2c3-c412f533dba1


Есть ли такие случаи, когда заявка существует, а покупка нет? Нет, проверка ниже

In [152]:
from collections import defaultdict 

In [162]:
# найти впервые оставленные заявки 
clients = leads_df["l_client_id"].unique()
clients = clients[clients != "00000000-0000-0000-0000-000000000000"]
counts = leads_df["l_client_id"].value_counts()
new_leads = defaultdict(list)
for client in clients:
    if counts[client] == 1: # если клиент встретился один раз, то считаем его заявку новой
        lead = leads_df[leads_df["l_client_id"] == client]
        for col in lead.columns:
            new_leads[col].append(lead[col].iloc[0])
    else:
        leads = leads_df[leads_df["l_client_id"] == client].sort_values(by="created_at") # иначе выбираем первую по времени
        for col in leads.columns:
            new_leads[col].append(leads[col].iloc[0])
            
new_leads = pd.DataFrame(new_leads)

In [170]:
# проверить, что клиенты до этих заявок не совершали покупок (не совершал никто, если брать для всего датасета)
for client_id in new_leads["l_client_id"]:
    if client_id in trans_df["l_client_id"]:
        print(client_id)

In [229]:
# значит, достаточно проверить только заявки, поскольку нет таких покупок, которые были совершены без заявки (проверка ниже)
def set_if_new(row):
    counts = leads_df["l_client_id"].value_counts()
    if row["l_client_id"] == "00000000-0000-0000-0000-000000000000":
        return "not new"
    if counts[row["l_client_id"]] == 1: # если клиент встретился один раз, то считаем его заявку новой
        return "new"
    # иначе проверим, соответствует ли id заявки первой заявке от этого клиента
    leads = leads_df[leads_df["l_client_id"] == row["l_client_id"]].sort_values(by="created_at")
    if row["lead_id"] == leads.iloc[0]["lead_id"]:
        return "new"
    else: return "not new"

In [230]:
leads_df["is_new"] = leads_df.apply(lambda x: set_if_new(x), axis=1)

#### 4. количество покупателей (кто купил в течение недели после заявки)

#### 5. количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)

In [231]:
#  функция, которая сразу определяет, совершил ли клиент покупку и был ли этот клиент новым
def set_if_buyer(row):
    client_id = row["l_client_id"]
    if client_id not in trans_df["l_client_id"].tolist():
        return "not buyer"
    
    transaction = trans_df[trans_df["l_client_id"] == client_id]
    row_dt = pd.to_datetime(row["created_at"], utc=True)
    transaction_dt = transaction.copy()
    transaction_dt["created_at"] = transaction["created_at"].apply(lambda x: pd.to_datetime(x, utc=True))
    transaction_dt["created_at"] = transaction_dt["created_at"].sort_values(ascending=False) 
    
    is_new = True # пока считаем, что покупатель новый
    for t_dt in transaction_dt["created_at"]:
        dt = (t_dt - row_dt).days # дата покупки минус дата продажи в днях
        if dt < 0 and is_new: # если обнаружили, что покупка уже была совершена до заявки, то покупателдь уже не новый
            is_new = False
        if dt >= 0 and dt <= 7:
            if is_new:
                return "new buyer"
            else:
                return "buyer"
    return "not buyer"

In [232]:
leads_df["is_buyer"] = leads_df.apply(lambda row: set_if_buyer(row), axis=1)

In [233]:
leads_df.head()

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,is_trash,is_buyer,is_new,m_real_amount
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,not trash,new buyer,new,44041
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,not trash,not buyer,new,0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,not trash,not buyer,new,0
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,trash,not buyer,not new,0
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,not trash,not buyer,new,0


#### 6. доход от покупок новых покупателей

In [234]:
# функция, которая подтягивает стоимость покупки
def set_trans_amount(row):
    client_id = row["l_client_id"]
    if client_id not in trans_df["l_client_id"].tolist():
        return 0
    
    transaction = trans_df[trans_df["l_client_id"] == client_id]
    row_dt = pd.to_datetime(row["created_at"], utc=True)
    transaction_dt = transaction.copy()
    transaction_dt["created_at"] = transaction["created_at"].apply(lambda x: pd.to_datetime(x, utc=True))
    transaction_dt["created_at"] = transaction_dt["created_at"].sort_values(ascending=False) 
    
    is_new = True # пока считаем, что покупатель новый
    for i, t_dt in enumerate(transaction_dt["created_at"]):
        dt = (t_dt - row_dt).days # дата покупки - дата продажи
        if dt >= 0 and dt <= 7: # если покупка была совершена (не факт, что она новая, это мы проверим в дальнейшем в сводной таблице)
            return transaction_dt.iloc[i]["m_real_amount"]
    return 0

In [235]:
leads_df["m_real_amount"] = leads_df.apply(lambda x: set_trans_amount(x), axis=1)

In [236]:
# запись на диск таблицы, на основе которой будет формироваться сводная
write_table(leads_df.drop(["created_at", "d_utm_medium", "d_utm_source", "l_manager_id", "l_client_id"], axis=1), "pivot!A1:J"+str(len(leads_df)+1))

{'spreadsheetId': '17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA', 'totalUpdatedRows': 3338, 'totalUpdatedColumns': 5, 'totalUpdatedCells': 16690, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '17UbRcA-UgY2CbqeumagIHnctQsqj5lu_iJUTMq_IFqA', 'updatedRange': 'pivot!A1:E3338', 'updatedRows': 3338, 'updatedColumns': 5, 'updatedCells': 16690}]}
